In [7]:
from datasets import load_dataset
import pandas as pd
import json
from openai import OpenAI

In [2]:
data = load_dataset("truthful_qa", "generation")

In [3]:
open_ai_format = [{"messages": [{"role": "system", "content": "You are a factual chatbot"},
    {"role": "user", "content": question},
    {"role": "assistant", "content": answer}]} for question, answer in zip(data['validation']['question'], data['validation']['best_answer'])]

In [14]:
with open('open_ai_fineTune.jsonl', 'w') as f:
    for line in open_ai_format[:10]:  # Shortening the number of lines to reduce cost
        f.write(json.dumps(line))
        f.write("\n")


In [10]:
from secretKey import secretKey

In [12]:
client = OpenAI(api_key = secretKey)

In [15]:
client.files.create(
  file=open("open_ai_fineTune.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-2vhGaKXu0SriawvB4ujFwYzG', bytes=2558, created_at=1714415867, filename='open_ai_fineTune.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [22]:
client.fine_tuning.jobs.create(
  training_file="file-2vhGaKXu0SriawvB4ujFwYzG", 
  model="gpt-3.5-turbo",
  hyperparameters = {"n_epochs": 2} #Shortening epochs to reduce cost
)

FineTuningJob(id='ftjob-JfiCHaxZ0TgQTVzz8W37vse4', created_at=1714416332, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KzH8syhAcjJ0Uusaj6ChfHyU', result_files=[], seed=37539524, status='validating_files', trained_tokens=None, training_file='file-2vhGaKXu0SriawvB4ujFwYzG', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None)

In [25]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9JQKHER7",
  messages=[
    {"role": "system", "content": "You are a factual chatbot."},
    {"role": "user", "content": "What would happen to a person if they ate an apple seed?"}
  ]
)
print(completion.choices[0].message.content)

Apple seeds contain amygdalin, which releases cyanide when it comes into contact with digestive enzymes. While small amounts of cyanide can be detoxified by the liver, eating a large amount of apple seeds can lead to cyanide poisoning.
